In [75]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
import sklearn
from sklearn.ensemble import GradientBoostingClassifier 
prefix='../'

In [2]:
def get_event(event):
    hits= pd.read_csv(prefix+'train_1/%s-hits.csv'%event)
    cells= pd.read_csv(prefix+'train_1/%s-cells.csv'%event)
    truth= pd.read_csv(prefix+'train_1/%s-truth.csv'%event)
    particles= pd.read_csv(prefix+'train_1/%s-particles.csv'%event)
    return hits, cells, truth, particles

In [3]:
# you can jump to step4 for test only.
train = True
if train:
    Train = []
    for i in tqdm_notebook(range(10,20)):
        event = 'event0000010%02d'%i
        hits, cells, truth, particles = get_event(event)
        hit_cells = cells.groupby(['hit_id']).value.count().values
        hit_value = cells.groupby(['hit_id']).value.sum().values
        features = np.hstack((hits[['x','y','z']]/1000, hit_cells.reshape(len(hit_cells),1)/10,hit_value.reshape(len(hit_cells),1)))
        particle_ids = truth.particle_id.unique()
        particle_ids = particle_ids[np.where(particle_ids!=0)[0]]

        pair = []
        for particle_id in particle_ids:
            hit_ids = truth[truth.particle_id == particle_id].hit_id.values-1
            for i in hit_ids:
                for j in hit_ids:
                    if i != j:
                        pair.append([i,j])
        pair = np.array(pair)   
        Train1 = np.hstack((features[pair[:,0]], features[pair[:,1]], np.ones((len(pair),1))))

        if len(Train) == 0:
            Train = Train1
        else:
            Train = np.vstack((Train,Train1))

        n = len(hits)
        size = len(Train1)*3
        p_id = truth.particle_id.values
        i =np.random.randint(n, size=size)
        j =np.random.randint(n, size=size)
        pair = np.hstack((i.reshape(size,1),j.reshape(size,1)))
        pair = pair[((p_id[i]==0) | (p_id[i]!=p_id[j]))]

        Train0 = np.hstack((features[pair[:,0]], features[pair[:,1]], np.zeros((len(pair),1))))

        print(event, Train1.shape)

        Train = np.vstack((Train,Train0))
    del Train0, Train1

    np.random.shuffle(Train)
    print(Train.shape)

event000001010 (905342, 11)
event000001011 (1049940, 11)
event000001012 (975962, 11)
event000001013 (937140, 11)
event000001014 (1138386, 11)
event000001015 (1096946, 11)
event000001016 (1054472, 11)
event000001017 (1125976, 11)
event000001018 (787588, 11)
event000001019 (1099946, 11)

(40684207, 11)


In [35]:
clf = GradientBoostingClassifier(max_depth=10)

In [36]:
%%time
clf.fit(Train[:200000,:-1], Train[:200000, -1])

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [37]:
%%time
clf.score(Train[-130000:-100000, :-1], Train[-130000:-100000, -1])

CPU times: user 162 ms, sys: 13.3 ms, total: 175 ms
Wall time: 185 ms


0.9535333333333333

#### Using 1000 trees

In [39]:
import sys
sys.getsizeof(Train)
with open('train10events.npy', 'wb') as file:
    np.save(file, Train)

In [87]:
clf2 = GradientBoostingClassifier(max_depth = None, n_estimators = 1000)

In [88]:
%%time
clf2.fit(Train[:100000,:-1], Train[:100000, -1])

CPU times: user 13min 23s, sys: 3.11 s, total: 13min 26s
Wall time: 13min 30s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [91]:
%%time
clf.score(Train[-150000:, :-1], Train[-150000:, -1])

CPU times: user 1min 21s, sys: 3.94 s, total: 1min 25s
Wall time: 1min 27s


0.9615268

In [90]:
clf.n_estimators

100

In [93]:
import pickle
with open('100_Gradient_Boost_96-5.p', 'wb') as file:
    pickle.dump(clf, file)